In [8]:
#login to hugging face

from huggingface_hub import notebook_login
notebook_login()

In [9]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "/Users/software/Downloads/hieroglyphics_nlp/language_model/hieroglyph_translation_t5_train.ipynb"


In [10]:
from datasets import Dataset
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import ast
import json

In [11]:
df = pd.read_csv("/Users/software/Downloads/hieroglyphics_nlp/language_model/hieroglyph_translation_dictionary_storage.csv")

In [12]:
def convert_to_dict(string):
    try:
        string = string.replace("'", '"')
        return json.loads(string)
    except json.JSONDecodeError as e:
        
        return None
df.drop("Unnamed: 0",axis=1)
df['Translations'] = df['Translations'].apply(convert_to_dict)
df.to_csv("hello.csv")

In [13]:
df.drop("Unnamed: 0",axis=1)

,Translations
0,"{'se': 'A1', 'de': 'I, me, my '}"
1,"{'se': 'A1 A1 A1', 'de': 'man, men, mankind, E..."
2,"{'se': 'A2', 'de': 'drink '}"
3,"{'se': 'A6', 'de': 'pure, purify oneself, bath..."
4,"{'se': 'A6A', 'de': 'pure, purify oneself, bat..."
...,...
11718,"{'se': 'Aa32 X1 G4 T14 A1 N33A', 'de': 'Nubian..."
11719,"{'se': 'Aa32 X1 N25', 'de': 'Nubia '}"
11720,"{'se': 'Aa32 X1 N25 T14 A1 N33A', 'de': 'Nubia..."
11721,"{'se': 'Aa32 X1 Z4 N33 Z2', 'de': 'Nubian mine..."


In [14]:

df["Translations"]

0                         {'se': 'A1', 'de': 'I, me, my '}
1        {'se': 'A1 A1 A1', 'de': 'man, men, mankind, E...
2                             {'se': 'A2', 'de': 'drink '}
3        {'se': 'A6', 'de': 'pure, purify oneself, bath...
4        {'se': 'A6A', 'de': 'pure, purify oneself, bat...
                               ...                        
11718    {'se': 'Aa32 X1 G4 T14 A1 N33A', 'de': 'Nubian...
11719                {'se': 'Aa32 X1 N25', 'de': 'Nubia '}
11720    {'se': 'Aa32 X1 N25 T14 A1 N33A', 'de': 'Nubia...
11721    {'se': 'Aa32 X1 Z4 N33 Z2', 'de': 'Nubian mine...
11722                 {'se': 'Aa32 X1 Z8', 'de': 'Nubia '}
Name: Translations, Length: 11723, dtype: object

In [15]:
# lets get our custom dataset in order

# Replace 'your_dataset.csv' with the actual file path of your CSV file
file_path = 'hieroglyph_translation_dictionary_storage_final.csv'

# Load your dataset into a Dataset object
hieroglyph_translation = Dataset.from_pandas(df)


# Print a sample from your dataset
print(hieroglyph_translation)



Dataset({
    features: ['Unnamed: 0', 'Translations'],
    num_rows: 11723
})


In [16]:
df1 = df[0:11000]
df2 = df[10723:11724]
print(df1.shape)
print(df2.shape)

(11000, 2)
(1000, 2)


In [17]:
train_ds = Dataset.from_pandas(df1, split="train")
test_ds = Dataset.from_pandas(df2, split="test")

In [18]:
print(train_ds[1]['Translations'])
type(train_ds[1]['Translations'])

{'de': 'man, men, mankind, Egyptians ', 'se': 'A1 A1 A1'}


dict

In [19]:
import json
type(hieroglyph_translation["Translations"][1])



dict

In [20]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [21]:
source_lang = "se"
target_lang = "de"
prefix = "Translate hieroglyph unicode sequence to English: "

def preprocess_function(examples):
    #try:
    inputs=[]
    targets=[]
    x=0
    for example in examples['Translations']:
        
        try:
            inputs.append(prefix+example[source_lang])
            targets.append(example[target_lang])
        except:
            inputs.append("None")
            targets.append("None")
            
        

    model_inputs = tokenizer(inputs, text_target=targets, max_length=1000, truncation=True)
    return model_inputs

    #except:
    #    print("Aiyayo Vada Poche")
    
tokenized_train_hieroglyph_translations = train_ds.map(preprocess_function, batched=True)

tokenized_test_hieroglyph_translations = test_ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/11000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [22]:
tokenized_test_hieroglyph_translations[0]

{'Unnamed: 0': 10723,
 'Translations': {'de': 'my ', 'se': 'X1 G1 M17 M17 A1'},
 'input_ids': [30355,
  15,
  1382,
  32,
  29929,
  73,
  23,
  4978,
  5932,
  12,
  1566,
  10,
  3,
  4,
  536,
  350,
  536,
  283,
  2517,
  283,
  2517,
  71,
  536,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [82, 1]}

In [23]:
tokenized_train_hieroglyph_translations[2]

{'Unnamed: 0': 2,
 'Translations': {'de': 'drink ', 'se': 'A2'},
 'input_ids': [30355,
  15,
  1382,
  32,
  29929,
  73,
  23,
  4978,
  5932,
  12,
  1566,
  10,
  71,
  357,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [3281, 1]}

In [24]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [25]:
import numpy as np
import evaluate
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [26]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="hieroglyph_10_epochs",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/AnushS/hieroglyph_10_epochs into local empty directory.
/Users/software/anaconda3/envs/mlp/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+

Error: An unexpected error occurred

In [ ]:
#trainer.push_to_hub()

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("AnushS/hieroglyph_unicode_translator_t5_small")
model = AutoModelForSeq2SeqLM.from_pretrained("AnushS/hieroglyph_unicode_translator_t5_small")

In [7]:
text = "Translate hieroglyph unicode sequence to English:  {hieroglyphs}"


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("AnushS/hieroglyph_unicode_translator_t5_small")
inputs = tokenizer(text, return_tensors="pt").input_ids

from transformers import AutoModelForSeq2SeqLM


model = AutoModelForSeq2SeqLM.from_pretrained("AnushS/hieroglyph_unicode_translator_t5_small")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
translated_keywords = str(tokenizer.decode(outputs[0], skip_special_tokens=True))


print(translated_keywords)


the body (of all)
